In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
from modelData import *
from gmsPython import nestingTree

No clean-up of work-folder


# Create Model Data

## 0. Preliminaries

*This creates a copy of the IO data from the ```IOdata``` project into the local data folder*

In [2]:
f_IOdata = os.path.join(d['project'], 'IOdata','data','processedData', 'IO_WCGE')
shutil.copy(f_IOdata, os.path.join(d['data'], 'IO_WCGE'))

'C:\\Users\\sxj477\\Documents\\GitHub\\CGE_Generator - Copy\\projects\\WasteCGE\\data\\IO_WCGE'

*Define main settings:*

In [3]:
t0 = 2019 # baseline year for the model
T  = t0 + 100 # set finite time horizon
name = 'version_A' # Some global name that carries through all models/data to identify this version

*Load data:*

In [4]:
db = GpyDB(os.path.join(d['data'], 'IO_WCGE'), name = f'{name}_{t0}')

*Clean up database - only include data for the baseline year $2019$:*

In [5]:
AggDB.subsetDB(db, pd.Index([t0], name = 't'));

*For mappings that identify nesting structure, we remove the time index entirely (we do not allow nesting structure to change over time here):*

In [6]:
[db.__setitem__(k, db(k).droplevel('t').unique()) for k in ['nEqui','d_qS','d_qD','d_qSEqui','dExport','dImport','dImport_dom','dImport_for']];

*Add various time subsets that we'll rely on:*

In [7]:
addTimeToDB(t0, T, db)

## 1. Production module

### 1.1. Nesting structure

Generally, we consider the following nesting structure:

In [8]:
mFull = pd.MultiIndex.from_tuples([('KELM', 'RxE'), ('KELM','KEL'), 
                                                    ('KEL','L'), ('KEL','KE'), 
                                                    ('KE','K'), ('KE','E'),
                                                    ('E','Energy_input'),('E','Energy_F')], names = ['n','nn'])

The materials nest excluding energy (```RxE```) is a CES nest of different final goods that compete with similar foreign ones. We use the syntax ```RxEym_x``` to denote the intermediate good that consists of ```x``` and the relevant foreign good:

In [9]:
n_noEnergy = db('n_p').difference(['Energy']) 
nF_noEnergy= db('n_F').difference(['Energy_F'])
RxE1 = pd.MultiIndex.from_product([['RxE'], 'RxEym_'+n_noEnergy], names = ['n','nn']) # mapping from RxE to intermediate goods
RxE2 = pd.MultiIndex.from_arrays([RxE1.get_level_values('nn'), n_noEnergy+'_input'], names = ['n','nn']) # mapping from intermediate good to domestic one 
RxE3 = pd.MultiIndex.from_arrays([RxE1.get_level_values('nn'), nF_noEnergy], names = ['n','nn']) # mapping from intermediate good to foreign one
mFull = reduce(pd.Index.union, [mFull, RxE1, RxE2, RxE3]) # Combine the four indices

If a sector uses inputs from all other sectors and foreign sectors, this is the mapping that describes its production structure. We allow for each sector to have a unique nesting structure by adding the sector index:

In [10]:
m = pyDatabases.cartesianProductIndex([db('s_p'), mFull])

Next, it may not be the case that all sectors rely on all types of inputs. In this case, we "trim" the nesting tree by eliminating the knots that are unused:

In [11]:
sparsity = adj.rc_pd(db('vD')[db('vD')!=0], db('s_p')).droplevel('t').index # what types of inputs do the sectors actually use in the data
# recall that inputs are called '_input' for products from the domestic sectors. Map the sparsity index to these:
mapNames = pd.Series(db('n_p')+'_input', index = db('n_p')).combine_first(
            pd.Series(sparsity.levels[-1], index = sparsity.levels[-1]))
sparsity = sparsity.set_levels(sparsity.levels[-1].map(mapNames), level = -1)

m = nestingTree.trimNestingStructure(m, sparsity)

Finally, we replace the upper-most level ```KELM``` with the name of the sector instead:

In [12]:
df = m.to_frame(index=False)
df.loc[df.n == 'KELM','n'] = df.loc[df.n == 'KELM', 's']
m = pd.MultiIndex.from_frame(df)
db['nestProduction'] = m

### 1.2. Elasticities

Elasticity of substitution between materials and ```qKEL```

In [13]:
sigmaOut = pd.Series(0.5, index = m[m.get_level_values('s')==m.get_level_values('n')].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```qKEL``` nest:

In [14]:
sigmaKEL = pd.Series(0.5, index  = m[m.get_level_values('n')=='KEL'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```KE``` nest:

In [15]:
sigmaKE = pd.Series(0.6, index  = m[m.get_level_values('n')=='KE'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```E``` nest:

In [16]:
sigmaE = pd.Series(0.5, index  = m[m.get_level_values('n')=='E'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution between materials:

In [17]:
sigmaRxE = pd.Series(0.1, index = m[m.get_level_values('n')=='RxE'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution import/domestic types:

In [18]:
sigmaRxEym = pd.Series(5, index = m[m.get_level_values('n').str.contains('RxEy')].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [19]:
sigma = pd.concat([sigmaOut, sigmaKEL, sigmaKE, sigmaE, sigmaRxE, sigmaRxEym], axis = 0 )
db.aom(sigma, name = 'sigma')

### 1.3. Regulation and emissions

Define relevant sector outputs and output shares (relevant if sectors have multiple outputs):

In [20]:
output = adj.rc_pd(db['qS'], db['s_p']) # output
inputs = adj.rc_pd(db['qD'], ('and', [db['s_p'], ('not', db['dur_p'])])) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares
db['qCO2'] = db('qCO2') * outShares # overwrite qCO2 to be split into potentially multiple outputs from each sector

Define implicit tax rate (this is a rough estimate of emission taxes, and its only available for $t\geq 1995$):

In [21]:
db['tauCO2'] = adjMultiIndex.applyMult((db('vTax').xs('Emissions',level='taxTypes') / db('qCO2').replace(0,1)).fillna(0), output.index)
db['dtauCO2']= db('tauCO2').index
db['dqCO2']  = db('qCO2').index

Add the rest of the regulation:
* Zero lump-sum taxes, zero input taxes.
* non-environmental output taxes adjusted to target total taxes

In [22]:
db.aom(adjMultiIndex.applyMult(adj.rc_pd(db('TotalTax'), db['s_p']), output.index) / output, name = 'tauS') 
db.aom(pd.Series(0, index = inputs.index), name = 'tauD') 
db.aom(pd.Series(0, index = adj.rc_pd(db('TotalTax'), db('s_p')).index), name = 'tauLump')
db.aom( (db('tauS')-db('vTax').xs('Emissions',level='taxTypes'))/output, name = 'tauNonEnv')
db.aom(db('tauNonEnv').copy(), name = 'tauNonEnv0') # exogenous variable used for calibration

Define sector-specific prices given regulation:

In [23]:
db.aom(stdSort(adj.rc_pd((db('tauD')+db('p')).dropna(), inputs)), name = 'pD')

## 2. Investment module

### 2.1. Nesting structure

The investment module consists only of the materials nest, but includes energy:

In [24]:
dImport = adj.rc_pd(db('dImport'), db('s_i'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_i'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_i'))

*Add mappings:*

In [25]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'Materials_'+x['n'].astype(str), u = 'Materials')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [26]:
df = dImport_dom.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [27]:
df = dImport_for.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level (Materials) with the name of the sector:

In [28]:
df = m.to_frame(index=False)
df.loc[df.n == 'Materials','n'] = df.loc[df.n == 'Materials', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [29]:
db['nestInvestment'] = m

### 2.2: Elasticities:

In [30]:
sigmaI = pd.Series(.5, index = m.droplevel('nn').unique(), name = 'sigma')
db.aom(sigmaI, name = 'sigma')

### 2.3. Regulation:

The investment sector does not itself emit emissions; however, investments are still tied to emissions through this sectors' reliance on inputs from other sectors (that are emission intensive). The emission regulation is filled in as a "production module" as above (but with zero emissions), in order to rely on a similar code.

In [31]:
output = adj.rc_pd(db['qS'], db['s_i']) # output
inputs = adj.rc_pd(db['qD'], db['s_i']) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares
db.aom((adj.rc_pd(db('qCO2'), db['s_i']) * outShares).fillna(0), name = 'qCO2')
db.aom(pd.Series(0, index = output.index, name = 'tauCO2'), name = 'tauCO2')

The rest of the regulation is standard: We assume that all taxes are paid as an output tax, there are no input/lump sum taxes:

In [32]:
db.aom(adjMultiIndex.applyMult(adj.rc_pd(db('TotalTax'), db['s_i']), output.index) / output, name = 'tauS') # tauS
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')
db.aom(pd.Series(0, index = adj.rc_pd(db('TotalTax'), db('s_i')).index), name = 'tauLump')
db.aom(adj.rc_pd(db('tauS'), db('s_i')).copy(), name = 'tauNonEnv')
db.aom(db('tauNonEnv').copy(), name = 'tauNonEnv0') # exogenous variable used for calibration

Define sector-specific prices given regulation:

In [33]:
db.aom(stdSort(adj.rc_pd(db('tauD')+db('p'), inputs)), name = 'pD')

## 3. Households

### 3.1. Nesting structure

The household consumption nesting structure is somewhat similar to the investment sectors, with the exception that the top nest is a consumption aggregate that captures intertemporal preferences for consumption smoothing.

In [34]:
dImport = adj.rc_pd(db('dImport'), db('s_HH'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_HH'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_HH'))

*Add mappings:*

In [35]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'C_'+x['n'].astype(str), u = 'C')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [36]:
df = dImport_dom.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [37]:
df = dImport_for.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [38]:
df = m.to_frame(index=False)
df.loc[df.n == 'C','n'] = 'C_'+df.loc[df.n == 'C', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [39]:
db['nestHH'] = m

Add mapping from consumption aggregate to labor:

In [40]:
db['L2C'] = pd.MultiIndex.from_arrays([db('s_HH'), pd.Index(['L']*len(db('s_HH')), name = 'n'), ('C_'+db('s_HH')).rename('nn')])

### 3.2. Elasticities:

Upper level nest:

In [41]:
sigma_HH_upper = pd.Series(0.5, index = m[m.get_level_values('n') == 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [42]:
sigma_HH_Import = pd.Series(2, index = m[m.get_level_values('n') != 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [43]:
sigma_HH = pd.concat([sigma_HH_upper, sigma_HH_Import], axis = 0) 
db.aom(sigma_HH, name = 'sigma')

### 3.3. Regulation:

Households do not directly emit anything (it all happens in the production process). Currently, we use a very simple type of regulation here:
* Households pay VAT (the size of which is included in the IO data); we impose this as a flat rate on all inputs.
* Set the labor income tax rate at 0.47 (roughly the average for 2018 data).
* We set the level of lump sum transfers at 40\% of labor income (roughly the case for 2018 data).
* Finally, *for now/temporarily*, adjust the value of *TotalTax* to ensure that the consumer has a balanced budget. If we used detailed data on regulation on households, we could drop this assumption.

In [44]:
output = adj.rc_pd(db('qS'), db['s_HH'])
inputs = adj.rc_pd(db('qD'), db['s_HH'])
rTransfer = 0.4004
rLaborTax = 0.478
rVAT = adj.rc_pd(db('vTax').xs('VAT',level='taxTypes'), db['s_HH']) / pyDatabases.pdSum(inputs, 'n')
db.aom(adjMultiIndex.applyMult(rVAT,inputs.index), name = 'tauD') # add flat rate VAT on all demand
db.aom(pd.Series(rLaborTax, index = output.index), name = 'tauS') # add a flat labor tax rate
db.aom(pyDatabases.pdSum(-output * rTransfer, 'n'), name = 'tauLump') # Adjust lump sum tax
db.aom(pyDatabases.pdSum(rVAT * inputs, 'n')+pyDatabases.pdSum(output, 'n') * (rLaborTax-rTransfer), name = 'TotalTax') # adhoc adjustment of 'TotalTax'

## 4. Government

### 4.1. Nesting structure

Government consumption is nested in a similar way as household consumption - and uses the same elasticities:

In [45]:
dImport = adj.rc_pd(db('dImport'), db('s_G'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_G'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_G'))

*Add mappings:*

In [46]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'GC_'+x['n'].astype(str), u = 'GC')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [47]:
df = dImport_dom.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [48]:
df = dImport_for.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [49]:
df = m.to_frame(index=False)
df.loc[df.n == 'GC','n'] = 'GC_'+df.loc[df.n == 'GC', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [50]:
db['nestG'] = m

### 4.2. Elasticities

Upper level nest:

In [51]:
sigma_G_upper = pd.Series(.9, index = m[m.get_level_values('n') == 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [52]:
sigma_G_Import = pd.Series(2, index = m[m.get_level_values('n') != 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [53]:
sigma_G = pd.concat([sigma_G_upper, sigma_G_Import], axis = 0) 
db.aom(sigma_G, name = 'sigma')

### 4.2. Regulation

The government regulates/taxes itself (because data says so). Because we use incomplete data on taxes (especially on household taxes/transfers), we make some adhoc adjustments here:
* Remove taxes on inventory (because the "inventory sector" is only used for completeness of the IO system).
* Set flat VAT tax rate on government consumption to target taxes from government sector.

In [54]:
inputs = adj.rc_pd(db('qD'), db('s_G'))
db['TotalTax'] = adj.rc_pd(db('TotalTax'), ('not', pd.Index(['itory'],name='s'))) # remove inventory taxes
db['d_TotalTax'] = db['TotalTax'].index # what sectors pay taxes
rVAT = adj.rc_pd(db('TotalTax'), db('s_G')) / pyDatabases.pdSum(inputs,'n') # define VAT rate of taxes
db.aom(adjMultiIndex.applyMult(rVAT,inputs.index), name = 'tauD') # add flat rate VAT on all demand

## 5. Trade

Set export elasticity to 7.5:

In [55]:
db.aom(pd.Series(7.5, index = db('dExport')), name='sigma')

## 6. Globals

Other parameters/variables used throughout:

In [56]:
db.aom(pd.Series(db('R_LR'), index = db('t')), name = 'Rrate', priority = 'first') # fix interest rate path at long run level

## Export

In [57]:
db.export(repo=d['data'])